In [2]:
import brightway2 as bw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
if "ecoinvent 3.6" not in bw.databases:
    link = "E:\ecoinvent3.6cutoffecoSpold02\datasets"
    ei36 = bw.SingleOutputEcospold2Importer(link, "ecoinvent 3.6", use_mp=False)
    ei36.apply_strategies()
    ei36.statistics()
    ei36.write_database()

In [ ]:
bw.databases

In [ ]:
eidb = bw.Database("ecoinvent 3.6")

In [ ]:
act_elec_prod = sorted(
    [
        act
        for act in eidb
        if "electricity production, hard coal" in act["name"]
        and "CN" in act["location"]
        and "at coal mine power plant" not in act["name"]
    ]
)
act_elec_prod

In [ ]:
ReCiPe = [
    method
    for method in bw.methods
    if "ReCiPe Midpoint (H) V1.13" in str(method)
    and "w/o LT" not in str(method)
    and "no LT" not in str(method)
    and "obsolete" not in str(method)
]
ReCiPe

In [ ]:
indicators = [ReCiPe[6], ReCiPe[7], ReCiPe[13], ReCiPe[15]]
indicators

In [ ]:
elec_prod_FU = []
for act in act_elec_prod:
    elec_prod_FU.append({act: 1})

elec_prod_FU

In [ ]:
bw.calculation_setups["power_plant"] = {"inv": elec_prod_FU, "ia": indicators}
power_plant_LCA = bw.MultiLCA("power_plant").results

In [ ]:
indict = []
for i in indicators:
    indict.append(i[2])

act_name = []
for act in act_elec_prod:
    act_name.append(act["location"])

power_plant_df = pd.DataFrame(power_plant_LCA, index=act_name, columns=indict)

In [ ]:
power_plant_df.GWP100.mean(), power_plant_df.PMFP.mean(), power_plant_df.TAP100.mean()

## Data preparation

### Emissions from coal consumption

In [3]:
power_plant = pd.read_excel(
    r"E:\tencent files\chrome Download\Research\LCA\LCA SOFC\BW2Import\power_plant.xlsx",
    sheet_name="2014",
    columns=0,
    index=0,
)
power_plant.head()

,总序号,省序号,厂序号,省份,电厂,区划代码,地级市/自治州,location,发电类别,年末发电设备容量\n（千瓦）,...,\tAsh [kg/kg],\tCarbon [kg/kg]\t,Sulfur [kg/kg]\t,底-灰比,发电量(MWh),CO2 emissions (kg/kWhe),SO2 emissions (kg/kWhe),PM emissions (kg/kWhe),脱硫设备,除尘
0,1.0,1.0,1.0,北京市,华润协鑫燃气热电厂,110000,北京市,CN-BJ,NaN,150000.0,...,NaN,NaN,NaN,0.7,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,1.0,2.0,北京市,北京正东电子动力集团有限公司（燃气）,110000,北京市,CN-BJ,NaN,119980.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,1.0,3.0,北京市,北京大唐高井发电厂,110000,北京市,Beijing,NaN,NaN,...,0.09,0.64,0.008,0.7,197598.0,10.443627,NaN,NaN,石灰石-石膏湿法,布袋除尘
3,4.0,1.0,4.0,北京市,华能北京热电有限责任公司,110000,北京市,Beijing,NaN,845000.0,...,0.09,0.64,0.008,0.7,445568.0,NaN,NaN,NaN,石灰石-石膏湿法,静电除尘
4,5.0,1.0,5.0,北京市,华能北京热电有限责任公司（燃机）,110000,北京市,CN-BJ,NaN,923400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [4]:
power_plant_coal = power_plant[power_plant["发电耗用原煤量\n（吨）"].notnull()]
power_plant_coal.head()

,总序号,省序号,厂序号,省份,电厂,区划代码,地级市/自治州,location,发电类别,年末发电设备容量\n（千瓦）,...,\tAsh [kg/kg],\tCarbon [kg/kg]\t,Sulfur [kg/kg]\t,底-灰比,发电量(MWh),CO2 emissions (kg/kWhe),SO2 emissions (kg/kWhe),PM emissions (kg/kWhe),脱硫设备,除尘
2,3.0,1.0,3.0,北京市,北京大唐高井发电厂,110000,北京市,Beijing,NaN,NaN,...,0.09,0.64,0.008,0.7,197598.0,10.443627,NaN,NaN,石灰石-石膏湿法,布袋除尘
3,4.0,1.0,4.0,北京市,华能北京热电有限责任公司,110000,北京市,Beijing,NaN,845000.0,...,0.09,0.64,0.008,0.7,445568.0,NaN,NaN,NaN,石灰石-石膏湿法,静电除尘
5,6.0,1.0,6.0,北京市,北京京能热电股份有限公司,110000,北京市,Beijing,NaN,600000.0,...,0.09,0.64,0.008,0.7,471101.0,NaN,NaN,NaN,石灰石-石膏湿法,静电除尘
9,10.0,1.0,10.0,北京市,国华北京热电分公司（一热）,110000,北京市,Beijing,NaN,400000.0,...,0.09,0.64,0.008,0.7,218303.0,NaN,NaN,NaN,石灰石-石膏湿法,静电除尘
18,19.0,2.0,2.0,天津市,天津军粮城发电厂,120000,天津市,Tianjin,NaN,800000.0,...,0.09,0.64,0.008,0.7,437343.0,NaN,NaN,NaN,石灰石-石膏湿法,电袋除尘


<IPython.core.display.Javascript object>

In [5]:
coal_rank = ["烟煤", "无烟煤", "贫煤"]
for row_position, row in zip(
    range(len(power_plant_coal["Coal rank"])), power_plant_coal["Coal rank"]
):
    if row not in coal_rank:
        power_plant_coal["Coal rank"].iloc[row_position] = "烟煤"

C:\Users\教务处13\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-5-1cbe8e26196e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  power_plant_coal["Coal rank"].iloc[row_position] = "烟煤"


<IPython.core.display.Javascript object>

In [6]:
for row in power_plant_coal.index:
    if power_plant_coal.loc[row, "Coal rank"] == coal_rank[0]:
        power_plant_coal.loc[row, "LHV[MJ/kg] "] = 26.3
        power_plant_coal.loc[row, "\tAsh [kg/kg] "] = 0.09
        power_plant_coal.loc[row, "\tCarbon [kg/kg]\t"] = 0.71
        power_plant_coal.loc[row, "Sulfur [kg/kg]\t"] = 0.0088
    if power_plant_coal.loc[row, "Coal rank"] == coal_rank[1]:
        power_plant_coal.loc[row, "LHV[MJ/kg] "] = 28.7
        power_plant_coal.loc[row, "\tAsh [kg/kg] "] = 0.11
        power_plant_coal.loc[row, "\tCarbon [kg/kg]\t"] = 0.81
        power_plant_coal.loc[row, "Sulfur [kg/kg]\t"] = 0.0079
    if power_plant_coal.loc[row, "Coal rank"] == coal_rank[2]:
        power_plant_coal.loc[row, "LHV[MJ/kg] "] = 27.5
        power_plant_coal.loc[row, "\tAsh [kg/kg] "] = 0.1
        power_plant_coal.loc[row, "\tCarbon [kg/kg]\t"] = 0.76
        power_plant_coal.loc[row, "Sulfur [kg/kg]\t"] = 0.00835

C:\Users\教务处13\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


<IPython.core.display.Javascript object>

In [7]:
LHV_sce = 29.307

for row in power_plant_coal[power_plant_coal["发电量\n（万千瓦时）"].notnull()].index:
    power_plant_coal.loc[row, "发电量(MWh)"] = (
        10 * power_plant_coal.loc[row, "发电量\n（万千瓦时）"]
    )
for row in power_plant_coal[power_plant_coal["发电量\n（万千瓦时）"].isnull()].index:
    power_plant_coal.loc[row, "发电量(MWh)"] = (
        1000
        * power_plant_coal.loc[row, "发电耗用原煤量\n（吨）"]
        * power_plant_coal.loc[row, "LHV[MJ/kg] "]
        / (LHV_sce * power_plant_coal.loc[row, "供电标准煤耗率\n（克/千瓦时）"])
    )

<IPython.core.display.Javascript object>

In [8]:
power_plant_coal.loc[:, "CO2 emissions (kg/kWhe)"] = (
    44
    / 12
    * (
        power_plant_coal.loc[:, "发电耗用原煤量\n（吨）"]
        * power_plant_coal.loc[:, "\tCarbon [kg/kg]\t"]
        / power_plant_coal.loc[:, "发电量(MWh)"]
    )
)
power_plant_coal.loc[:, "SO2 emissions (kg/kWhe)"] = (
    64
    / 32
    * (
        power_plant_coal.loc[:, "发电耗用原煤量\n（吨）"]
        * power_plant_coal.loc[:, "Sulfur [kg/kg]\t"]
        / power_plant_coal.loc[:, "发电量(MWh)"]
    )
)
power_plant_coal.loc[:, "PM emissions (kg/kWhe)"] = (
    0.063  # size distribution of PM2.5
    * 64
    / 32
    * (
        power_plant_coal.loc[:, "发电耗用原煤量\n（吨）"]
        * power_plant_coal.loc[:, "\tAsh [kg/kg] "]
        * power_plant_coal.loc[:, "底-灰比"]
        / power_plant_coal.loc[:, "发电量(MWh)"]
    )
)

<IPython.core.display.Javascript object>

In [9]:
len(power_plant_coal)

1571

<IPython.core.display.Javascript object>

发现有一些电厂的排放值异常，考虑可能存在数据错误，将这些outlier从数据中剔除

In [10]:
power_plant_coal_screen =  power_plant_coal[
        power_plant_coal["CO2 emissions (kg/kWhe)"]
        <= power_plant_coal["CO2 emissions (kg/kWhe)"].quantile(0.95)
    ]


<IPython.core.display.Javascript object>

In [11]:
power_plant_coal_screen.to_excel(r"d:\desktop\power_plant_emissions1.xlsx")

<IPython.core.display.Javascript object>

### Flue gas treatment

SO2 removal efficiency

In [12]:
so2_removal_df = pd.read_excel(
    r"E:\tencent files\chrome Download\Research\LCA\LCA SOFC\BW2Import\power_plant.xlsx",
    sheet_name="desulfurization",
    names=["name_CN", "name_EN", "removal rate", "comment"],
)

<IPython.core.display.Javascript object>

In [13]:
so2_removal_dic = dict(zip(so2_removal_df["name_CN"], so2_removal_df["removal rate"]))

<IPython.core.display.Javascript object>

In [14]:
power_plant_coal_screen.head(2)

,总序号,省序号,厂序号,省份,电厂,区划代码,地级市/自治州,location,发电类别,年末发电设备容量\n（千瓦）,...,\tAsh [kg/kg],\tCarbon [kg/kg]\t,Sulfur [kg/kg]\t,底-灰比,发电量(MWh),CO2 emissions (kg/kWhe),SO2 emissions (kg/kWhe),PM emissions (kg/kWhe),脱硫设备,除尘
2,3.0,1.0,3.0,北京市,北京大唐高井发电厂,110000,北京市,Beijing,NaN,NaN,...,0.09,0.71,0.0088,0.7,1975980.0,1.158590,0.007833,0.003533,石灰石-石膏湿法,布袋除尘
3,4.0,1.0,4.0,北京市,华能北京热电有限责任公司,110000,北京市,Beijing,NaN,845000.0,...,0.09,0.71,0.0088,0.7,4455680.0,0.828135,0.005599,0.002525,石灰石-石膏湿法,静电除尘


<IPython.core.display.Javascript object>

In [15]:
for row in power_plant_coal_screen.index:
    if power_plant_coal_screen.loc[row, "脱硫设备"] not in so2_removal_dic.keys():
        power_plant_coal_screen.loc[row, "脱硫设备"] = "石灰石-石膏湿法"

<IPython.core.display.Javascript object>

In [16]:
for row in power_plant_coal_screen.index:
    power_plant_coal_screen.loc[row, "so2RemovalRate"] = so2_removal_dic[
        power_plant_coal_screen.loc[row, "脱硫设备"]
    ]

C:\Users\教务处13\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


<IPython.core.display.Javascript object>

PM removal efficiency

In [17]:
PM_removal_df = pd.read_excel(
    r"E:\tencent files\chrome Download\Research\LCA\LCA SOFC\BW2Import\power_plant.xlsx",
    sheet_name="PM removal",
    names=["name_CN", "name_EN", "removal rate", "comment"],
)

<IPython.core.display.Javascript object>

In [18]:
PM_removal_dic = dict(zip(PM_removal_df["name_CN"], PM_removal_df["removal rate"]))
PM_removal_dic

{'静电除尘': 92.31,
 '布袋除尘': 99.3,
 '电袋除尘': 99.3,
 '湿式电除尘': 92.31,
 '静电/布袋': 99.3,
 '静电/电袋': 99.3,
 '水膜除尘': 98.5}

<IPython.core.display.Javascript object>

In [19]:
for row in power_plant_coal_screen.index:
    if power_plant_coal_screen.loc[row, "除尘"] not in PM_removal_dic.keys():
        power_plant_coal_screen.loc[row, "除尘"] = "静电除尘"

<IPython.core.display.Javascript object>

In [20]:
for row in power_plant_coal_screen.index:
    power_plant_coal_screen.loc[row, "PMRemovalRate"] = PM_removal_dic[
        power_plant_coal_screen.loc[row, "除尘"]
    ]

<IPython.core.display.Javascript object>

In [21]:
power_plant_coal_screen.loc[:, "CO2 emissions (kg)"] = (
    1000
    * 44
    / 12
    * (
        power_plant_coal_screen.loc[:, "发电耗用原煤量\n（吨）"]
        * power_plant_coal_screen.loc[:, "\tCarbon [kg/kg]\t"]
    )
)
power_plant_coal_screen.loc[:, "SO2 emissions (kg)"] = (
    1000
    * 64
    / 32
    * (
        power_plant_coal_screen.loc[:, "发电耗用原煤量\n（吨）"]
        * power_plant_coal_screen.loc[:, "Sulfur [kg/kg]\t"]
        * (1 - power_plant_coal_screen.loc[:, "so2RemovalRate"] / 100)
    )
)
power_plant_coal_screen.loc[:, "PM emissions (kg)"] = (
    1000
    * 0.063  # size distribution of PM2.5
    * 64
    / 32
    * (
        power_plant_coal_screen.loc[:, "发电耗用原煤量\n（吨）"]
        * power_plant_coal_screen.loc[:, "\tAsh [kg/kg] "]
        * power_plant_coal_screen.loc[:, "底-灰比"]
        * (1 - power_plant_coal_screen.loc[:, "PMRemovalRate"] / 100)
    )
)

<IPython.core.display.Javascript object>

### Impact of upstream coal supply

We now calculate the impact of coal_fired power plant on fossil depletion. To do this, we need the impact factors of different coal ranks in ecoinvent

In [22]:
coal_supply = [
    act
    for act in eidb
    if "market for hard coal" in act["name"] and "CN" in act["location"]
][0]
coal_supply

NameError: name 'eidb' is not defined

<IPython.core.display.Javascript object>

In [ ]:
coal_supply_FU = []
for coal_consume in power_plant_coal_screen.loc[:, "发电耗用原煤量\n（吨）"]:
    coal_consume_kg = coal_consume * 1000
    coal_supply_FU.append({coal_supply: coal_consume_kg})

In [ ]:
len(coal_supply_FU)

In [ ]:
bw.calculation_setups["coal_supply"] = {"inv": coal_supply_FU, "ia": indicators}
coal_supply_LCA = bw.MultiLCA("coal_supply").results

In [ ]:
coal_supply_df = pd.DataFrame(
    coal_supply_LCA, index=power_plant_coal_screen.index, columns=indict
)
coal_supply_df.head()

In [ ]:
power_plant_total = pd.concat([power_plant_coal_screen, coal_supply_df], axis=1)
power_plant_total.head(2)

In [ ]:
power_plant_total.loc[:, "GWP_total"] = (
    power_plant_total.loc[:, "CO2 emissions (kg)"] + power_plant_total.loc[:, "GWP100"]
)
power_plant_total.loc[:, "FDP_total"] = power_plant_total.loc[:, "FDP"]
power_plant_total.loc[:, "PMFP_total"] = (
    power_plant_total.loc[:, "PM emissions (kg)"] + power_plant_total.loc[:, "PMFP"]
)
power_plant_total.loc[:, "AP_total"] = (
    power_plant_total.loc[:, "SO2 emissions (kg)"] + power_plant_total.loc[:, "TAP100"]
)

In [ ]:
elec_generation = 1000 * power_plant_coal.loc[:, "发电量(MWh)"]
power_plant_total.loc[:, "GWP/kWh"] = (
    power_plant_total.loc[:, "GWP_total"] / elec_generation
)
power_plant_total.loc[:, "FDP/kWh"] = (
    power_plant_total.loc[:, "FDP_total"] / elec_generation
)
power_plant_total.loc[:, "PMFP/kWh"] = (
    power_plant_total.loc[:, "PMFP_total"] / elec_generation
)
power_plant_total.loc[:, "AP/kWh"] = (
    power_plant_total.loc[:, "AP_total"] / elec_generation
)

### Impact allocation 

In [ ]:
for row in power_plant_total.index:
    if "热" in power_plant_total.loc[row, "电厂"]:
        if power_plant_total.loc[row, "年末发电设备容量\n（千瓦）"] <= 50000:
            power_plant_total.loc[row, "Heat allocation"] = 0.61128 / (3.6 + 0.61128)
        elif 50000 < power_plant_total.loc[row, "年末发电设备容量\n（千瓦）"] <= 200000:
            power_plant_total.loc[row, "Heat allocation"] = (0.61128 * 0.5) / (
                3.6 + (0.5 * 0.61128)
            )
        else:
            power_plant_total.loc[row, "Heat allocation"] = (0.61128 * 0.5 / 3) / (
                3.6 + (0.5 * 0.61128) / 3
            )
    else:
        power_plant_total.loc[row, "Heat allocation"] = 0

In [ ]:
ls = ["GWP/kWh", "FDP/kWh", "PMFP/kWh", "AP/kWh"]
for impact in ls:
    power_plant_total.loc[:, impact + "allocated"] = power_plant_total.loc[
        :, impact
    ] * (1 - power_plant_total.loc[:, "Heat allocation"])

In [ ]:
power_plant_total

## Geocoding

In [42]:
import requests
import time
import json
import hashlib
from urllib import parse

<IPython.core.display.Javascript object>

In [43]:
def GetBaiduResponse(addtress):
    # 以get请求为例http://api.map.baidu.com/geocoder/v2/?address=百度大厦&output=json&ak=你的ak
    queryStr = (
        "/geocoding/v3/?address=%s&output=json&ak=fWGr3QvW9doZ7wYcgG7gGLo1zH9Fm83a"
        % addtress
    )
    # 对queryStr进行转码，safe内的保留字符不转换
    encodedStr = parse.quote(queryStr, safe="/:=&?#+!$,;'@()*[]")
    # 在最后直接追加上yoursk
    rawStr = encodedStr + "x4NuvzXZGW641qzz6TxGjwHh5fy1qoQ9"
    # 计算sn
    sn = hashlib.md5(parse.quote_plus(rawStr).encode("utf8")).hexdigest()
    # 由于URL里面含有中文，所以需要用parse.quote进行处理，然后返回最终可调用的url
    url = parse.quote(
        "http://api.map.baidu.com" + queryStr + "&sn=" + sn, safe="/:=&?#+!$,;'@()*[]"
    )
    res = requests.get(url, headers={"content-type": "application/json"})

    return res.content.decode()

<IPython.core.display.Javascript object>

In [44]:
coord_ls = []
for row in power_plant_total.index:
    try:
        coord = (
            eval(GetBaiduResponse(power_plant_total.loc[row, "电厂"]))["result"][
                "location"
            ]["lng"],
            eval(GetBaiduResponse(power_plant_total.loc[row, "电厂"]))["result"][
                "location"
            ]["lat"],
        )
        coord_ls.append(coord)
    except:
        coord_ls.append("ValueError")

<IPython.core.display.Javascript object>

In [45]:
coord_ls

[(116.36363203458903, 39.984881072602),
 (116.54208994618455, 39.889441615372384),
 (116.15326397568477, 39.93047204551698),
 (116.49002577577615, 39.91495797627284),
 (117.4535433250982, 39.064037010000206),
 (117.05970570815823, 39.15730908519974),
 (117.22309795838483, 39.13807010367603),
 (117.19857817050666, 38.965947043430205),
 (116.98046860454797, 38.95337068917225),
 (117.2095232146708, 39.093667843403956),
 (117.40729155066155, 39.13327350550781),
 (117.70242758020103, 39.06249069512622),
 (117.4706539789606, 39.98169906719955),
 (117.33546735395016, 39.094095013394885),
 (119.82762105908833, 33.99564654746655),
 (117.74608962968296, 38.95140350586373),
 (117.7294160912817, 38.9449293136214),
 (117.66727203474585, 39.00508152300104),
 (117.01797627315047, 39.22225242956271),
 (117.17825364902122, 39.124989452129974),
 (117.50014597766705, 38.98036409294963),
 (117.36340402649661, 39.225313082291244),
 (117.94343630106735, 39.22365001063983),
 (117.2095232146708, 39.0936678434

<IPython.core.display.Javascript object>

In [46]:
coord_df = pd.DataFrame(coord_ls).iloc[:, :2]
coord_df1 = coord_df.rename(
    index=dict(zip(coord_df.index, power_plant_total.index)),
    columns={0: "longitude", 1: "latitude"},
)

<IPython.core.display.Javascript object>

In [47]:
coord_df1

,longitude,latitude
2,116.364,39.9849
3,116.542,39.8894
5,116.153,39.9305
9,116.49,39.915
18,117.454,39.064
...,...,...
7878,116.337,39.7685
7936,86.9051,44.1977
7937,81.2874,40.5533
7941,116.283,39.9213


<IPython.core.display.Javascript object>

In [48]:
power_plant_total_with_coor = pd.concat([power_plant_total, coord_df1], axis=1)

<IPython.core.display.Javascript object>

Some power plants cannot find a proper address by Baidu API, so we use the coordination of their cities.

In [49]:
for row in power_plant_total_with_coor.index:
    if power_plant_total_with_coor.loc[row, "longitude"] == "V":
        power_plant_total_with_coor.loc[row, "longitude"] = eval(
            GetBaiduResponse(power_plant_total.loc[row, "地级市/自治州"])
        )["result"]["location"]["lng"]
        power_plant_total_with_coor.loc[row, "latitude"] = eval(
            GetBaiduResponse(power_plant_total.loc[row, "地级市/自治州"])
        )["result"]["location"]["lat"]

<IPython.core.display.Javascript object>

In [50]:
power_plant_total_with_coor.to_excel(
    r"E:\tencent files\chrome Download\Research\LCA\LCA SOFC\output\coal_power_plant.xlsx"
)

<IPython.core.display.Javascript object>